This Query is created by usersohamjcpp

In [2]:
%% sql
```
WITH Premia_Pcom_customer AS (
    SELECT *
    FROM Premia
    WHERE table_name = 'Pcom_customer'
),
Bancs_T_RPDB_POL_PARTY AS (
    SELECT *
    FROM Bancs
    WHERE table_name = 'T_RPDB_POL_PARTY'
),
Dim_intermediary_type AS (
    SELECT DISTINCT LEFT(cust_code,2) AS intermediary_code
    FROM Premia_Pcom_customer
    WHERE cust_code LIKE '%%' || dim_intermediary_type || '%%'
),
Policy_Master_Agg AS (
    SELECT *
    FROM policy_master_agg
),
Derived AS (
    SELECT
        CASE
            WHEN (CURRENT_DATE - aml_acknowledged_timestamp) >= 365 THEN 'Yes'
            WHEN (CURRENT_DATE - aml_acknowledged_timestamp) < 365 THEN 'No'
        END AS due_aml_training
    FROM vo_login
)
SELECT
    Premia_Pcom_customer.cust_code AS intermediary_code,
    Premia_Pcom_customer.cust_name AS intermediary_name,
    Policy_Master_Agg.intermediary_type,
    Policy_Master_Agg.agent_mobile_no,
    Policy_Master_Agg.agent_email_id,
    vo_login.is_aml_acknowledged,
    vo_login.aml_acknowledged_timestamp,
    Derived.due_aml_training,
    DATE_SUB(CURRENT_DATE, INTERVAL 1 DAY) AS reported_date,
    CURRENT_TIMESTAMP AS load_timestamp,
    agg_policy_master_report_daily.fufiller_code,
    agg_policy_master_report_daily.fulfiller_name,
    agg_policy_master_report_daily.office_code,
    agg_policy_master_report_daily.office_name,
    agg_policy_master_report_daily.zone_name,
    agg_policy_master_report_daily.mkt_zone_name
FROM Premia_Pcom_customer
LEFT JOIN Bancs_T_RPDB_POL_PARTY ON Premia_Pcom_customer.cust_code = Bancs_T_RPDB_POL_PARTY.premia_cust_code
LEFT JOIN Dim_intermediary_type ON LEFT(Premia_Pcom_customer.cust_code,2) = Dim_intermediary_type.intermediary_code
LEFT JOIN Policy_Master_Agg ON Premia_Pcom_customer.intermediary_code = Policy_Master_Agg.intermediary_code
LEFT JOIN vo_login ON Premia_Pcom_customer.cust_code = vo_login.cust_code
LEFT JOIN Derived ON Derived.aml_acknowledged_timestamp = vo_login.aml_acknowledged_timestamp
LEFT JOIN agg_policy_master_report_daily ON Premia_Pcom_customer.intermediary_code = agg_policy_master_report_daily.intermediary_code
ORDER BY agg_policy_master_report_daily.policy_from_date DESC
LIMIT 100;
```